# Process Raw RX Data

The raw_rx_data folder contains the data logged by the receiver during testing and the timing data for each test setup. This notebook processes that data to label the TX brightness levels (min, mid, max) and selects the valid data points (received signals inbetween test setups are discarded).

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

Select data file and test setup timing file:|

In [2]:
data_path = 'raw_rx_data/LOG_23_corridor_ir_led.csv'
timing_path = 'raw_rx_data/timing_corridor_ir_led.csv'

In [3]:
data = pd.read_csv(data_path)

data['diff'] = data['local'] - data['received']         #calculate time difference in milliseconds
data['ts'] = pd.to_datetime(data['local'], unit='ms')
data.set_index('ts', inplace=True)

In [4]:
data.head()

,local,received,diff,correction,header_duration0,header_duration1,high_min,high_max,low_short_min,low_short_max,low_long_min,low_long_max,high_avg,low_short_avg,low_long_avg
ts,,,,,,,,,,,,,,,
2020-06-01 10:03:10.453,1591005790453,1591005790424,29,80,7209,3610,432,480,404,472,1349,1377,446,444,1359
2020-06-01 10:03:10.980,1591005790980,1591005790931,49,87,7192,3627,398,478,407,495,1350,1398,423,470,1374
2020-06-01 10:03:11.455,1591005791455,1591005791427,28,83,7239,3559,456,502,401,449,1305,1334,480,414,1317
2020-06-01 10:03:12.455,1591005792455,1591005792427,28,83,7210,3609,433,479,422,471,1335,1375,446,445,1356
2020-06-01 10:03:12.957,1591005792957,1591005792929,28,85,7235,3564,453,526,379,451,1306,1355,476,419,1319


Label each received timestamp with the transmit brigthness (min, mid,max). Transmit pattern: min - 0.5s - mid - 0.5s - max - 1s

In [5]:
i = 0
min_ms = data['local'][i]
mid_ms = data['local'][i+1]
max_ms = data['local'][i+2]
data['brightness'] = ""
data.loc[(((data['local'] - min_ms) % 2000 - 1000).abs() - 1000).abs() < 100, 'brightness'] = 'min'
data.loc[(((data['local'] - mid_ms) % 2000 - 1000).abs() - 1000).abs() < 100, 'brightness'] = 'mid'
data.loc[(((data['local'] - max_ms) % 2000 - 1000).abs() - 1000).abs() < 100, 'brightness'] = 'max'

Get start and end time of each test setup. Align start and end time so that an equal number of timestamps is transmitted at each brightness for a given test setup.

In [6]:
timing = pd.read_csv(timing_path)
timing.dropna(inplace=True)
timing['aligned_start_ms'] = timing['start time'] + 2000 - ((timing['start time'] - min_ms) % 2000) - 100
timing['aligned_end_ms'] = timing['end time'] - ((timing['end time'] - max_ms) % 2000) + 100
timing['start_ts'] = pd.to_datetime(timing['aligned_start_ms'], unit='ms')
timing['end_ts'] = pd.to_datetime(timing['aligned_end_ms'], unit='ms')
timing

,location,light,orientation,distance,start time,end time,Unnamed: 6,aligned_start_ms,aligned_end_ms,start_ts,end_ts
0,corridor,lamp + 10mA IR,0,0.5,1591006678271,1591006739463,61,1591006678353,1591006739555,2020-06-01 10:17:58.353,2020-06-01 10:18:59.555
1,corridor,lamp + 10mA IR,0,1.0,1591006763148,1591006830009,67,1591006764353,1591006829555,2020-06-01 10:19:24.353,2020-06-01 10:20:29.555
2,corridor,lamp + 10mA IR,0,2.0,1591006849188,1591006913890,65,1591006850353,1591006913555,2020-06-01 10:20:50.353,2020-06-01 10:21:53.555
3,corridor,lamp + 10mA IR,0,3.0,1591006933334,1591006997701,64,1591006934353,1591006997555,2020-06-01 10:22:14.353,2020-06-01 10:23:17.555
4,corridor,lamp + 10mA IR,0,4.0,1591007014025,1591007080867,67,1591007014353,1591007079555,2020-06-01 10:23:34.353,2020-06-01 10:24:39.555
5,corridor,lamp + 10mA IR,0,5.0,1591007099703,1591007173895,74,1591007100353,1591007173555,2020-06-01 10:25:00.353,2020-06-01 10:26:13.555
6,corridor,lamp + 50mA IR,0,0.5,1591006583120,1591006645357,62,1591006584353,1591006643555,2020-06-01 10:16:24.353,2020-06-01 10:17:23.555
7,corridor,lamp + 50mA IR,0,1.0,1591006502643,1591006564269,62,1591006504353,1591006563555,2020-06-01 10:15:04.353,2020-06-01 10:16:03.555
8,corridor,lamp + 50mA IR,0,2.0,1591006409169,1591006475297,66,1591006410353,1591006473555,2020-06-01 10:13:30.353,2020-06-01 10:14:33.555
9,corridor,lamp + 50mA IR,0,3.0,1591006322666,1591006391426,69,1591006324353,1591006389555,2020-06-01 10:12:04.353,2020-06-01 10:13:09.555


Classify received timestamps as correct if the absolute difference is less than 200 ms.

In [7]:
data['correct'] = data['diff'].abs() < 200

Save processed data for evaluation.

In [8]:
data.to_csv('processed_data/corridor_ir_led.csv')